# Model to Classify Elo Customer Loyalty Outliers

_Note! If you want to commit any changes to this document, please strip all output (Cell > Current Outputs > Clear, or set up [nbstripout](https://github.com/kynan/nbstripout) as a git filter) from this notebook before doing so. Thanks!_

For more detailed descriptions of some of these steps, see the `elo_loyalty_prediction` notebook.

PS. For now this model is not very successful when it comes to predicting outliers. Perhaps we can come back to it when we have done more feature engineering.

## Load Libraries and Data

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
from fastai import *
from fastai.tabular import *
from fastai.metrics import *

In [ ]:
train_and_validation_df = pd.read_csv('data/processed/train_with_aggregated_features.csv', index_col='card_id')
test_df = pd.read_csv('data/processed/test_with_aggregated_features.csv', index_col='card_id')

## Split Into Train and Validation Sets

In [ ]:
from sklearn.model_selection import train_test_split
train_df, validate_df = train_test_split(train_and_validation_df, test_size=0.2, random_state=238923)

In [ ]:
train_df.head()

## The Issue

If we have a look at the loyalty scores that we want to predict for this competition, we'll see that there are a bunch of outliers at around -30 loyalty. (NB. this field is likely normalised to have mean 0 and standard deviation 1, meaning these outliers are probably 0 \[nan?] in the original data set.)

If we could disregard these, our model for predicting the loyalty score would have a much easier time. But of course we don't know which of the incoming fields are outliers in this sense and which aren't. So let's try to make a classifier model to predict whether or not a sample (a `card_id`) is an outlier!

In [ ]:
sns.distplot(train_df.target)

Let's call any sample with a loyalty score below -25 an outlier.

In [ ]:
train_df['is_outlier'] = train_df.target < -25
validate_df['is_outlier'] = validate_df.target < -25

In [ ]:
sns.countplot(x='is_outlier', data=train_df)

## Set Up Model

Again, for more detailed comments on some of these steps, have a look at the `elo_loyalty_prediction` notebook.

We will do some upsampling of the outliers in the training set – basically copying those rows so that they don't drown among all the countless non-outlier samples.

In [ ]:
upsampled_train_df = train_df.copy().append([train_df[train_df.is_outlier]] * 10, ignore_index=True)

In [ ]:
valid_idx = range(len(upsampled_train_df), len(upsampled_train_df) + len(validate_df)); valid_idx

In [ ]:
category_names = ['first_active_monthYear',
                  'first_active_monthMonth',
                  'first_active_monthWeek',
                  'first_active_monthIs_quarter_start',
                  'first_active_monthIs_year_start',
                  'feature_1',
                  'feature_2',
                  'feature_3',
                  'authorized_flag_top',
                  'category_1_transaction_top',
                  'category_1_merchant_top',
                  'category_2_top',
                  'category_3_top',
                  'category_4_top',
                  'subsector_id_transaction_top',
                  'subsector_id_merchant_top',
                  'city_id_top',
                  'state_id_top',
                  'purchase_Year_top',
                  'purchase_Month_top',
                  'purchase_Week_top',
                  'purchase_Day_top',
                  'purchase_Dayofweek_top',
                  'most_recent_sales_range_top',
                  'most_recent_purchases_range_top',
                  'merch_authorized_flag_top',
                  'merch_category_1_transaction_top',
                  'merch_category_1_merchant_top',
                  'merch_category_2_top',
                  'merch_category_3_top',
                  'merch_category_4_top',
                  'merch_subsector_id_transaction_top',
                  'merch_subsector_id_merchant_top',
                  'merch_city_id_top',
                  'merch_state_id_top',
                  'merch_purchase_Year_top',
                  'merch_purchase_Month_top',
                  'merch_purchase_Week_top',
                  'merch_purchase_Day_top',
                  'merch_purchase_Dayofweek_top',
                  'merch_most_recent_sales_range_top',
                  'merch_most_recent_purchases_range_top',]
continuous_names = ['first_active_monthElapsed',
                    'purchase_amount_sum',
                    'purchase_amount_mean',
                    'purchase_amount_min',
                    'purchase_amount_max',
                    'purchase_amount_std',
                    'installments_sum',
                    'installments_mean',
                    'installments_min',
                    'installments_max',
                    'installments_std',
                    'month_lag_mean',
                    'month_lag_min',
                    'month_lag_max',
                    'merchant_id_nunique',
                    'state_id_nunique',
                    'city_id_nunique',
                    'numerical_1_sum',
                    'numerical_1_mean',
                    'numerical_1_min',
                    'numerical_1_max',
                    'numerical_1_std',
                    'numerical_2_sum',
                    'numerical_2_mean',
                    'numerical_2_min',
                    'numerical_2_max',
                    'numerical_2_std',
                    'avg_sales_lag3_sum',
                    'avg_sales_lag3_mean',
                    'avg_sales_lag3_min',
                    'avg_sales_lag3_max',
                    'avg_sales_lag3_std',
                    'avg_sales_lag6_sum',
                    'avg_sales_lag6_mean',
                    'avg_sales_lag6_min',
                    'avg_sales_lag6_max',
                    'avg_sales_lag6_std',
                    'avg_sales_lag12_sum',
                    'avg_sales_lag12_mean',
                    'avg_sales_lag12_min',
                    'avg_sales_lag12_max',
                    'avg_sales_lag12_std',
                    'avg_purchases_lag3_sum',
                    'avg_purchases_lag3_mean',
                    'avg_purchases_lag3_min',
                    'avg_purchases_lag3_max',
                    'avg_purchases_lag3_std',
                    'avg_purchases_lag6_sum',
                    'avg_purchases_lag6_mean',
                    'avg_purchases_lag6_min',
                    'avg_purchases_lag6_max',
                    'avg_purchases_lag6_std',
                    'avg_purchases_lag12_sum',
                    'avg_purchases_lag12_mean',
                    'avg_purchases_lag12_min',
                    'avg_purchases_lag12_max',
                    'avg_purchases_lag12_std',
                    'active_months_lag3_sum',
                    'active_months_lag3_mean',
                    'active_months_lag3_min',
                    'active_months_lag3_std',
                    'active_months_lag6_sum',
                    'active_months_lag6_mean',
                    'active_months_lag6_min',
                    'active_months_lag6_std',
                    'active_months_lag12_sum',
                    'active_months_lag12_mean',
                    'active_months_lag12_min',
                    'active_months_lag12_max',
                    'active_months_lag12_std',
                    'merchant_category_id_transaction_nunique',
                    'merchant_category_id_merchant_nunique',
                    'subsector_id_transaction_nunique',
                    'subsector_id_merchant_nunique',
                    'merchant_group_id_nunique',
                    'most_recent_sales_range_nunique',
                    'most_recent_purchases_range_nunique',
                    'elapsed_since_last_purchase_sum',
                    'elapsed_since_last_purchase_mean',
                    'elapsed_since_last_purchase_min',
                    'elapsed_since_last_purchase_max',
                    'elapsed_since_last_purchase_std',
                    'elapsed_since_last_merch_purchase_sum',
                    'elapsed_since_last_merch_purchase_mean',
                    'elapsed_since_last_merch_purchase_min',
                    'elapsed_since_last_merch_purchase_max',
                    'elapsed_since_last_merch_purchase_std',
                    'authorized_flag_Y_ratio',
                    'category_1_transaction_N_ratio',
                    'category_1_merchant_N_ratio',
                    'category_2_1.0_ratio',
                    'category_2_3.0_ratio',
                    'category_2_4.0_ratio',
                    'category_2_2.0_ratio',
                    'category_2_5.0_ratio',
                    'category_3_A_ratio',
                    'category_3_B_ratio',
                    'category_3_C_ratio',
                    'category_4_N_ratio',
                    'purchase_Is_month_start_True_ratio',
                    'purchase_Is_month_end_False_ratio',
                    'purchase_Year_2017_ratio',
                    'most_recent_sales_range_B_ratio',
                    'most_recent_sales_range_A_ratio',
                    'most_recent_sales_range_C_ratio',
                    'most_recent_sales_range_D_ratio',
                    'most_recent_sales_range_E_ratio',
                    'most_recent_purchases_range_B_ratio',
                    'most_recent_purchases_range_C_ratio',
                    'most_recent_purchases_range_A_ratio',
                    'most_recent_purchases_range_D_ratio',
                    'most_recent_purchases_range_E_ratio',
                    'merch_purchase_amount_sum',
                    'merch_purchase_amount_mean',
                    'merch_purchase_amount_min',
                    'merch_purchase_amount_max',
                    'merch_purchase_amount_std',
                    'merch_installments_sum',
                    'merch_installments_mean',
                    'merch_installments_min',
                    'merch_installments_max',
                    'merch_installments_std',
                    'merch_month_lag_mean',
                    'merch_month_lag_min',
                    'merch_month_lag_max',
                    'merch_merchant_id_nunique',
                    'merch_state_id_nunique',
                    'merch_city_id_nunique',
                    'merch_numerical_1_sum',
                    'merch_numerical_1_mean',
                    'merch_numerical_1_min',
                    'merch_numerical_1_max',
                    'merch_numerical_1_std',
                    'merch_numerical_2_sum',
                    'merch_numerical_2_mean',
                    'merch_numerical_2_min',
                    'merch_numerical_2_max',
                    'merch_numerical_2_std',
                    'merch_avg_sales_lag3_sum',
                    'merch_avg_sales_lag3_mean',
                    'merch_avg_sales_lag3_min',
                    'merch_avg_sales_lag3_max',
                    'merch_avg_sales_lag3_std',
                    'merch_avg_sales_lag6_sum',
                    'merch_avg_sales_lag6_mean',
                    'merch_avg_sales_lag6_min',
                    'merch_avg_sales_lag6_max',
                    'merch_avg_sales_lag6_std',
                    'merch_avg_sales_lag12_sum',
                    'merch_avg_sales_lag12_mean',
                    'merch_avg_sales_lag12_min',
                    'merch_avg_sales_lag12_max',
                    'merch_avg_sales_lag12_std',
                    'merch_avg_purchases_lag3_sum',
                    'merch_avg_purchases_lag3_mean',
                    'merch_avg_purchases_lag3_min',
                    'merch_avg_purchases_lag3_max',
                    'merch_avg_purchases_lag3_std',
                    'merch_avg_purchases_lag6_sum',
                    'merch_avg_purchases_lag6_mean',
                    'merch_avg_purchases_lag6_min',
                    'merch_avg_purchases_lag6_max',
                    'merch_avg_purchases_lag6_std',
                    'merch_avg_purchases_lag12_sum',
                    'merch_avg_purchases_lag12_mean',
                    'merch_avg_purchases_lag12_min',
                    'merch_avg_purchases_lag12_max',
                    'merch_avg_purchases_lag12_std',
                    'merch_active_months_lag3_sum',
                    'merch_active_months_lag3_mean',
                    'merch_active_months_lag3_min',
                    'merch_active_months_lag3_max',
                    'merch_active_months_lag3_std',
                    'merch_active_months_lag6_sum',
                    'merch_active_months_lag6_mean',
                    'merch_active_months_lag6_min',
                    'merch_active_months_lag6_max',
                    'merch_active_months_lag6_std',
                    'merch_active_months_lag12_sum',
                    'merch_active_months_lag12_mean',
                    'merch_active_months_lag12_min',
                    'merch_active_months_lag12_max',
                    'merch_active_months_lag12_std',
                    'merch_merchant_category_id_transaction_nunique',
                    'merch_merchant_category_id_merchant_nunique',
                    'merch_subsector_id_transaction_nunique',
                    'merch_subsector_id_merchant_nunique',
                    'merch_merchant_group_id_nunique',
                    'merch_most_recent_sales_range_nunique',
                    'merch_most_recent_purchases_range_nunique',
                    'merch_elapsed_since_last_purchase_sum',
                    'merch_elapsed_since_last_purchase_mean',
                    'merch_elapsed_since_last_purchase_min',
                    'merch_elapsed_since_last_purchase_max',
                    'merch_elapsed_since_last_purchase_std',
                    'merch_category_1_transaction_N_ratio',
                    'merch_category_1_merchant_N_ratio',
                    'merch_category_2_1.0_ratio',
                    'merch_category_2_4.0_ratio',
                    'merch_category_2_2.0_ratio',
                    'merch_category_2_3.0_ratio',
                    'merch_category_2_5.0_ratio',
                    'merch_category_3_B_ratio',
                    'merch_category_3_A_ratio',
                    'merch_category_3_C_ratio',
                    'merch_category_4_Y_ratio',
                    'merch_category_4_N_ratio',
                    'merch_purchase_Is_month_start_True_ratio',
                    'merch_purchase_Is_month_end_False_ratio',
                    'merch_purchase_Year_2017_ratio',
                    'merch_purchase_Year_2018_ratio',
                    'merch_most_recent_sales_range_A_ratio',
                    'merch_most_recent_sales_range_B_ratio',
                    'merch_most_recent_sales_range_D_ratio',
                    'merch_most_recent_sales_range_E_ratio',
                    'merch_most_recent_sales_range_C_ratio',
                    'merch_most_recent_purchases_range_A_ratio',
                    'merch_most_recent_purchases_range_B_ratio',
                    'merch_most_recent_purchases_range_D_ratio',
                    'merch_most_recent_purchases_range_C_ratio',
                    'merch_most_recent_purchases_range_E_ratio',]
dep_var = 'is_outlier'

In [ ]:
df = pd.concat([upsampled_train_df, validate_df]).reset_index()[category_names + continuous_names + [dep_var]]

In [ ]:
data = TabularDataBunch.from_df('data/unzipped',
                                df,
                                dep_var,
                                valid_idx=valid_idx,
                                procs=[FillMissing, Categorify, Normalize],
                                cat_names=category_names,
                                cont_names=continuous_names)

In [ ]:
data.show_batch()

In [ ]:
learn = tabular_learner(data,
                        layers=[200, 100],
                        ps=[1e-1, 5e-1],
                        emb_drop=0.04,
                        metrics=[accuracy, Precision(), Recall()])

In [ ]:
learn.model

## Train Model

In [ ]:
learn.lr_find()

In [ ]:
learn.recorder.plot()

In [ ]:
learn.fit_one_cycle(3, 1e-2, wd=0.8)

In [ ]:
learn.recorder.plot_losses()

## Make Predictions

Now that we have trained our model, lets make some predictions to see whether or not our metrics lie to us.

Let's only take those predictions where the model was at least 80% confident that the sample is an outlier, to increase our precision at the expense of recall.

In [ ]:
predictions, targets = [x.numpy() for x in learn.get_preds(DatasetType.Valid)]
# Each element in prediction is an array of two values, the likelihood of 
# `False` (not an outlier) and the likelihood of `True` (an outlier).
outlier_predictions = [x[1] > 0.8 for x in predictions]
outlier_targets = targets == 1

In [ ]:
prediction_df = pd.DataFrame({'prediction': outlier_predictions, 'target': outlier_targets})

In [ ]:
prediction_df.head()

In [ ]:
prediction_df.prediction.value_counts()

In [ ]:
prediction_df.target.value_counts()

In [ ]:
prediction_df[prediction_df.prediction].head()

Calculate **precision** _(fraction of relevant instances among the retrieved instances)_ and **recall** _(fraction of relevant instances that have been retrieved over the total amount of relevant instances)_.

In [ ]:
prediction_counts = prediction_df[prediction_df.prediction].target.value_counts()
false_positives = prediction_counts[0]
true_positives = prediction_counts[1]
precision = true_positives / (false_positives + true_positives); precision

In [ ]:
total_positives = prediction_df.target.value_counts()[1]
true_positives = prediction_counts[1]
recall = true_positives / total_positives; recall